In [435]:
import pandas as pd
import os
import gzip
import spacy as sp
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from collections import defaultdict
from sklearn.cluster import DBSCAN
from math import log
from nltk.corpus import wordnet as wn
import nltk
from itertools import product
import warnings
warnings.filterwarnings('ignore')

In [449]:
def similar_words(wordx, wordy):
    sem1, sem2 = wn.synsets(wordx), wn.synsets(wordy)
    for i,j in list(product(*[sem1,sem2])):
        if i == j:
            return True
    return False



In [44]:
nlp = sp.load('en')

In [417]:
SCORE_WEIGHTS = [0.7, 0.05 ,0.289, 0.603, 10]
CANDIDATE_MOST_COMMON_NUMBER = 200
total_count = -1


In [397]:
eval_data = pd.read_csv("eval-mod.csv")

metaphor_words = []
metaphors = {}

for r in eval_data.iterrows():
    tokens = nlp(r[1]["Metaphor"].decode('utf-8', errors='ignore').strip().lower())
    topic = tokens[0]
    vehicle = tokens[-1]
    if topic.lemma_ not in metaphor_words:
        metaphor_words.append(topic.lemma_)
    if vehicle.lemma_ not in metaphor_words:
        metaphor_words.append(vehicle.lemma_)
    if r[1]["Metaphor"] not in metaphors:
        metaphors[r[1]["Metaphor"]] = [topic.lemma_, vehicle.lemma_, 
                                       [(str(r[1]["Interpretation"]).lower(), r[1]["Freq"])],
                                       [str(r[1]["Interpretation"]).lower()]
                                      ]
    else:
        metaphors[r[1]["Metaphor"]][2].append((str(r[1]["Interpretation"]).lower(), r[1]["Freq"]))
        metaphors[r[1]["Metaphor"]][3].append(str(r[1]["Interpretation"]).lower())
print metaphors

{'education is a stairway': [u'education', u'stairway', [('higher', 4), ('steps', 3), ('upward', 3), ('long', 2), ('passage', 2), ('ascension', 2), ('climbing', 2), ('path', 2), ('success', 2), ('tiring', 2)], ['higher', 'steps', 'upward', 'long', 'passage', 'ascension', 'climbing', 'path', 'success', 'tiring']], 'obligation is a shackle': [u'obligation', u'shackle', [('annoying', 5), ('restraining', 4), ('metallic', 2), ('prison', 2), ('duty', 2), ('restricting', 2), ('trapped', 2), ('uncomfortable', 2)], ['annoying', 'restraining', 'metallic', 'prison', 'duty', 'restricting', 'trapped', 'uncomfortable']], 'lawyers are snakes': [u'lawyer', u'snake', [('sneaky', 7), ('liars', 5), ('slippery', 4), ('deceivable', 4), ('poisonous', 4), ('scary', 4), ('dangerous', 3), ('slithery', 2), ('evil', 2), ('ugly', 2), ('mean', 2), ('vicious', 2)], ['sneaky', 'liars', 'slippery', 'deceivable', 'poisonous', 'scary', 'dangerous', 'slithery', 'evil', 'ugly', 'mean', 'vicious']], 'deserts are ovens': [

In [162]:
class Lang:
    def __init__(self):
        self.vec = []
        self.word_count = 0
        self.ind2word = {}
        self.word2ind = {}
        for line in open("../glove.6B/glove.6B.300d.txt"):
            values = line.split(" ")
            v = []
            for i in range (1, len(values)):
                v.append(float(values[i]))
            self.vec.append(v)
            self.ind2word[self.word_count] = values[0]
            self.word2ind[values[0]] = self.word_count
            self.word_count += 1
    
    def get_vec(self, word):
        word = word.strip().lower()
        if word in self.word2ind:
            return self.vec[self.word2ind[word]]
        return None
            
lang = Lang()

In [371]:
class DepNode:
    def __init__(self, line):
        tokens = line.split('\t')
        self.id = int(tokens[0])
        self.text = tokens[1].lower()
        self.lemma = tokens[2].lower()
        self.pos = tokens[3]
        self.head = int(tokens[6])
        self.dep_type = tokens[7]
        self.children = []

class DepSentence:
    def __init__(self):
        self.nodes = []
        
    def add_word(self, line):
        node = DepNode(line)
        self.nodes.append(node)
        return node
        
    def rewire(self):
        for n in self.nodes:
            if n.dep_type != 'ROOT':
                self.nodes[n.head].children.append(n)
    
    def is_valid_meaning(self):
        #if self.pos in ['JJ', 'JJS', 'RB', 'VBG']:
        if self.pos in ['JJ', 'JJS', 'VBG']:
            return True
        return False
    
    
def find_candidates_for(words, dep_corpus_folder):
    candidates = {}
    files = [f for f in os.listdir(dep_corpus_folder) if os.path.isfile(os.path.join(dep_corpus_folder, f))]
    for f in files:
        full_path = os.path.join(dep_corpus_folder, f)
        print "processing", full_path
        sentence = None
        valid = False
        for line in gzip.open(full_path):
            line = line.strip()
            if len(line) > 0 and line[0] != '#':
                if sentence is None:
                    sentence = DepSentence()
                node = sentence.add_word(line)
                if node.text in words:
                    valid = True
            elif sentence is not None:
                if valid:
                    sentence.rewire()
                    for n in sentence.nodes:
                        if n.text in words:
                            if n.text not in candidates:
                                candidates[n.text] = Counter()
                            if n.dep_type != 'ROOT' and is_valid_meaning(sentence.nodes[n.head].pos):
                                candidates[n.text][sentence.nodes[n.head].text] += 1
                            for c in n.children:
                                if is_valid_meaning(c.pos):
                                    candidates[n.text][c.text] += 1
                sentence = None
    return candidates

In [372]:
candidates = find_candidates_for(metaphor_words, 'dep_corpus')

processing dep_corpus/part-m-00000.gz
processing dep_corpus/part-m-00001.gz
processing dep_corpus/part-m-00002.gz
processing dep_corpus/part-m-00003.gz
processing dep_corpus/part-m-00004.gz
processing dep_corpus/part-m-00005.gz
processing dep_corpus/part-m-00006.gz
processing dep_corpus/part-m-00007.gz
processing dep_corpus/part-m-00008.gz
processing dep_corpus/part-m-00009.gz
processing dep_corpus/part-m-00010.gz
processing dep_corpus/part-m-00011.gz
processing dep_corpus/part-m-00012.gz
processing dep_corpus/part-m-00013.gz
processing dep_corpus/part-m-00014.gz
processing dep_corpus/part-m-00015.gz
processing dep_corpus/part-m-00016.gz
processing dep_corpus/part-m-00017.gz
processing dep_corpus/part-m-00018.gz


In [373]:
print candidates['god'].most_common(200)

[('good', 333), ('not', 297), ('only', 228), ('true', 116), ('loving', 102), ('alone', 98), ('same', 89), ('angry', 75), ('great', 74), ('willing', 72), ('personal', 71), ("n't", 70), ('dear', 70), ('alive', 68), ('almighty', 67), ('all-powerful', 65), ('dead', 61), ('real', 54), ('just', 51), ('other', 50), ('faithful', 49), ('even', 48), ('then', 47), ('no', 44), ('merciful', 43), ('powerful', 41), ('so', 39), ('ever-living', 39), ('much', 38), ('such', 37), ('perfect', 37), ('omniscient', 37), ('present', 37), ('able', 35), ('omnipotent', 35), ('new', 34), ('false', 34), ('own', 34), ('holy', 34), ('infinite', 31), ('eternal', 28), ('also', 28), ('there', 28), ('responsible', 26), ('first', 24), ('perhaps', 24), ('amazing', 23), ('jealous', 23), ('human', 23), ('particular', 21), ('now', 21), ('best', 21), ('supernatural', 21), ('different', 21), ('happy', 20), ('awful', 19), ('christian', 19), ('therefore', 19), ('awesome', 19), ('wrong', 19), ('fully', 19), ('evil', 19), ('big', 1

In [384]:
def calculate_sem_score(candidate, metaphor_component):
    #metaphor_component can be either the topic or the vehicle
    cand_vec = lang.get_vec(candidate)
    meta_comp_vec = lang.get_vec(metaphor_component)
    if meta_comp_vec is None:
        print metaphor_component, "vector doesn't exist"
        return -1
    if cand_vec is None:
        return -1
    return cosine_similarity(cand_vec, meta_comp_vec)

total_count = -1

def calculate_total_candidate_count():
    global total_count
    if total_count > -1:
        return total_count
    
    count = 0
    for key in candidates:
        for v in candidates[key]:
            count += candidates[key][v]
    total_count = count
    return count


candidate_counts_cache = {}

def candidate_count(candidate):
    if candidate in candidate_counts_cache:
        return candidate_counts_cache[candidate]
    
    count = 0
    for key in candidates:
        count += candidates[key][candidate]
    candidate_counts_cache[candidate] = count   #add to cache
    return count


def candidate_component_npmi(candidate, metaphor_component):
    if candidate in candidates[metaphor_component]:
        c_x_y = candidates[metaphor_component][candidate]+1
    else:
        c_x_y = 1
    p_x_y = float(c_x_y) / float(calculate_total_candidate_count())
    c_x = candidate_count(candidate)
    p_x = float(c_x) / float(calculate_total_candidate_count())
    c_y = sum(candidates[metaphor_component].values())
    p_y = float(c_y) / float(calculate_total_candidate_count())
    pmi = log(p_x_y / (p_x * p_y))
    h_x_y = -log(p_x_y)
    npmi = pmi / float(h_x_y)
    return npmi
    

def calculate_final_score(candidate, topic, vehicle, weights):
    topic_sem_score = calculate_sem_score(candidate, topic)
    vehicle_sem_score = calculate_sem_score(candidate, vehicle)
    topic_npmi = candidate_component_npmi(candidate, topic)
    vehicle_npmi = candidate_component_npmi(candidate, vehicle)
    candidate_freq = candidate_count(candidate) / float(calculate_total_candidate_count())
    scores = [topic_sem_score, vehicle_sem_score, topic_npmi, vehicle_npmi, candidate_freq]
    #log linear score
    final_score = 0
    for i in range(0, 5):
        final_score += (weights[i] * (scores[i]))
    #return final_score, scores
    return final_score

In [473]:
def rank(topic, vehicle, candidate_most_common_number):
    if vehicle in candidates:
        most_vehicle_common = candidates[vehicle].most_common(candidate_most_common_number)
    else:
        most_vehicle_common = []
#     if topic in candidates:
#         most_topic_common = candidates[topic].most_common(candidate_most_common_number)
#     else:
    most_topic_common = []
    scores = defaultdict(float)
    for c in most_vehicle_common:
        c = c[0]
        try:
            if len(wn.synsets(c)) > 0:
                scores[c] = calculate_final_score(c, topic, vehicle, SCORE_WEIGHTS)
        except UnicodeDecodeError, e:
            print "error parsing ", c
    for c in most_topic_common:
        c = c[0]
        try:
            if len(wn.synsets(c)) > 0:
                scores[c] = calculate_final_score(c, topic, vehicle, SCORE_WEIGHTS)
        except UnicodeDecodeError, e:
            print "error parsing ", c
    
    final_before_clustering = []
    rank_id = 1
    for c in sorted(scores, key=scores.get, reverse=True):
        #print rank_id, c, scores[c]
        rank_id = rank_id + 1
        final_before_clustering.append(c)
    
    #return final_before_clustering
    #clustering
    vectors = []
    final_before_clustering_filtered = []
    for x in final_before_clustering:
        v = lang.get_vec(x)
        if v is not None:
            vectors.append(v)
            final_before_clustering_filtered.append(x)
    labels = DBSCAN(eps=2, min_samples=1).fit_predict(vectors)

    final_after_clustering = []
    clusters_seen_so_far = []
    clusters = {}
    for x in range(0, len(final_before_clustering_filtered)):
        candidate = final_before_clustering_filtered[x]
        label = labels[x]
        if label != -1 and label not in clusters_seen_so_far:
            clusters_seen_so_far.append(label)
            final_after_clustering.append(candidate)
            clusters[label] = [candidate]
        elif label == -1:
            final_after_clustering.append(candidate)
        else:
            clusters[label].append(candidate)
    #print clusters
    return final_after_clustering

def print_rank_results(res):
    for i in range(0, len(res)):
        print (i + 1), res[i]

In [475]:
print_rank_results(rank("alcohol", "crutch", CANDIDATE_MOST_COMMON_NUMBER))

In [476]:

MAX_CANDIDATES = 100

def interpretation_equals(int1, int2):
    return similar_words(int1, int2)


def exists_in_interpretation_list(candidate, interpretations, min_freq):
    for i in interpretations:
        if i[1] >= min_freq and interpretation_equals(i[0], candidate):
            return i[0]
    return None


def get_valid_interpretation_number(interpretations, min_freq):
    c = 0
    for i in interpretations:
        if i[1] >= min_freq:
            c += 1
    return c


def interpretation_in(candidate, interpretations):
    for i in interpretations:
        if interpretation_equals(i, candidate):
            return True
    return False


def eval():
    MRR = 0
    recall_at = {5:0.0, 10:0.0, 15:0.0, 25:0.0, 50:0.0}
    recall_at_participants = 0
    MAP = 0
    
    for m in metaphors:
        print m, metaphors[m][0], metaphors[m][1]
        pred_interpretations = rank(metaphors[m][0], metaphors[m][1], CANDIDATE_MOST_COMMON_NUMBER)
        MRR_added = False
        interpretations_seen_so_far = []
        valid_interpretation_number = get_valid_interpretation_number(metaphors[m][2], 5)
        if valid_interpretation_number > 0:
            recall_at_participants += 1
        relevants = 0
        curr_MAP = 0
        for i in range(0, min(MAX_CANDIDATES, len(pred_interpretations))):
            pi = pred_interpretations[i]
            #if pi in metaphors[m][3]:
            if interpretation_in(pi, metaphors[m][3]):
                if not MRR_added:
                    MRR += (1/float(i + 1))
                    MRR_added = True
                relevants += 1
                prec_at_k = float(relevants) / float(i + 1)
                curr_MAP += float(prec_at_k)
            if valid_interpretation_number > 0:
                inter = exists_in_interpretation_list(pi, metaphors[m][2], 5)
                if inter != None and inter not in interpretations_seen_so_far:
                    interpretations_seen_so_far.append(inter)
                if i in recall_at:
                    recall_at[i] += (float(len(interpretations_seen_so_far)) / float(valid_interpretation_number))
        if not MRR_added:
            MRR += 0
        if relevants > 0:
            curr_MAP /= float(relevants)
        MAP += curr_MAP
    MRR /= float(len(metaphors))
    MAP /= float(len(metaphors))
    results = []
    for i in sorted(recall_at):
        results.append(recall_at[i] / float(recall_at_participants))
    results.insert(0, MAP)    
    results.insert(0, MRR)
    return results

res = eval()
print('\t'.join([str(x) for x in res]))

education is a stairway education stairway
obligation is a shackle obligation shackle
lawyers are snakes lawyer snake
deserts are ovens desert oven
error parsing  °
error parsing  400º
error parsing  350º
error parsing  350ºf
error parsing  350Ëš
family is a fortress family fortress
knowledge is money knowledge money
television is a candy television candy
error parsing  crème
science is politics science politic
runner is a torpedo runner torpedo
error parsing  zoë
heart is a closet heart closet
alcohol is a crutch alcohol crutch
knowledge is a light knowledge light
love is a child love child
friendship is a rainbow friendship rainbow
women is a cat woman cat
life is a bottle life bottle
love is melody love melody
error parsing  naïve
desk is a junkyard desk junkyard
school is a zoo school zoo
city is a jungle city jungle
winter is a death winter death
error parsing  β-cell
eyelids are curtains eyelid curtain
teacher is a sculptor teacher sculptor
skating is a flying skating fly
knowled